## Maximum path sum I

Problem 18

By starting at the top of the triangle below and moving to adjacent numbers on the row below, the maximum total from top to bottom is 23.

3
7 4
2 4 6
8 5 9 3

That is, 3 + 7 + 4 + 9 = 23.

Find the maximum total from top to bottom of the triangle below:

75  
95 64  
17 47 82  
18 35 87 10  
20 04 82 47 65  
19 01 23 75 03 34  
88 02 77 73 07 63 67  
99 65 04 28 06 16 70 92  
41 41 26 56 83 40 80 70 33  
41 48 72 33 47 32 37 16 94 29  
53 71 44 65 25 43 91 52 97 51 14  
70 11 33 28 77 73 17 78 39 68 17 57  
91 71 52 38 17 14 91 43 58 50 27 29 48  
63 66 04 68 89 53 67 30 73 16 69 87 40 31  
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23  

NOTE: As there are only 16384 routes, it is possible to solve this problem by trying every route. However, Problem 67, is the same challenge with a triangle containing one-hundred rows; it cannot be solved by brute force, and requires a clever method! ;o)


Link: https://projecteuler.net/problem=18

Date solved:  
2022/03/08

In [1]:
# imports
import sys
sys.path.append('../../modules')

import parsing
import graphs

In [8]:
# %%timeit # results = 77.2 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


triangle = '''75  
95 64  
17 47 82  
18 35 87 10  
20 04 82 47 65  
19 01 23 75 03 34  
88 02 77 73 07 63 67  
99 65 04 28 06 16 70 92  
41 41 26 56 83 40 80 70 33  
41 48 72 33 47 32 37 16 94 29  
53 71 44 65 25 43 91 52 97 51 14  
70 11 33 28 77 73 17 78 39 68 17 57  
91 71 52 38 17 14 91 43 58 50 27 29 48  
63 66 04 68 89 53 67 30 73 16 69 87 40 31  
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23  '''


array = parsing.read_array_str(triangle)

graph = graphs.Graph()
n0 = graphs.Node()
previous_nodes = [n0]
for i,row in enumerate(array[:-1]):
    next_nodes=[]
    for j,edge in enumerate(row):
        new_node = graphs.Node()
        next_nodes.append(new_node)




    print(i,row)
    



        



0 75
0 [75]
0 95
1 64
1 [95, 64]
0 17
1 47
2 82
2 [17, 47, 82]
0 18
1 35
2 87
3 10
3 [18, 35, 87, 10]
0 20
1 4
2 82
3 47
4 65
4 [20, 4, 82, 47, 65]
0 19
1 1
2 23
3 75
4 3
5 34
5 [19, 1, 23, 75, 3, 34]
0 88
1 2
2 77
3 73
4 7
5 63
6 67
6 [88, 2, 77, 73, 7, 63, 67]
0 99
1 65
2 4
3 28
4 6
5 16
6 70
7 92
7 [99, 65, 4, 28, 6, 16, 70, 92]
0 41
1 41
2 26
3 56
4 83
5 40
6 80
7 70
8 33
8 [41, 41, 26, 56, 83, 40, 80, 70, 33]
0 41
1 48
2 72
3 33
4 47
5 32
6 37
7 16
8 94
9 29
9 [41, 48, 72, 33, 47, 32, 37, 16, 94, 29]
0 53
1 71
2 44
3 65
4 25
5 43
6 91
7 52
8 97
9 51
10 14
10 [53, 71, 44, 65, 25, 43, 91, 52, 97, 51, 14]
0 70
1 11
2 33
3 28
4 77
5 73
6 17
7 78
8 39
9 68
10 17
11 57
11 [70, 11, 33, 28, 77, 73, 17, 78, 39, 68, 17, 57]
0 91
1 71
2 52
3 38
4 17
5 14
6 91
7 43
8 58
9 50
10 27
11 29
12 48
12 [91, 71, 52, 38, 17, 14, 91, 43, 58, 50, 27, 29, 48]
0 63
1 66
2 4
3 68
4 89
5 53
6 67
7 30
8 73
9 16
10 69
11 87
12 40
13 31
13 [63, 66, 4, 68, 89, 53, 67, 30, 73, 16, 69, 87, 40, 31]


## ANSWER = 142913828922